%md
# Steam Project for the bloc 2 of Jedha certification  
# ------------------------------- 
# Part 2 : Analysis at macro level  

Author : Youenn PATAT

This notebook is the 2nd part, with the macro level analysis.


In [0]:
spark

sc = spark.sparkContext

In [0]:
filepath = "/Users/youenn.patat@gmail.com/Project_Steam/Steam_data_dataset_zoomed_1.json"

df = spark.read.json(filepath)

In [0]:
from pyspark.sql import functions as F # This will load the class where spark sql functions are contained
from pyspark.sql import Row # this will let us manipulate rows with spark sql
from functools import reduce
from pyspark.sql.functions import when

In [0]:
display(df.limit(10))

appid,categories,ccu,developer,discount,genre,header_image,initialprice,languages,name,negative,owners,platforms,positive,price,publisher,release_date,required_age,short_description,tags,type,website
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,https://cdn.akamai.steamstatic.com/steam/apps/10/header.jpg?t=1666823513,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000","List(true, true, true)",201215,999,Valve,2000/11/1,0,Play the world's number 1 online action game. Engage in an incredibly realistic brand of terrorist warfare in this wildly popular team-based game. Ally with teammates to complete strategic missions. Take out enemy sites. Rescue hostages. Your role affects your team's success. Your team's success affects your role.,"List(266, 1191, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 5426, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 227, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 2784, null, null, null, null, null, null, null, null, null, null, null, null, 1607, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 4831, null, null, null, null, null, null, null, null, null, 1707, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 632, null, null, null, null, null, null, null, null, null, null, null, 3392, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 131, null, null, 769, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 881, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 289, null, null, null, 3353, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 614, null, null, null, null, null, null, 304, null, null, null, 1344, null, null, 1864, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, null, 1192)",game,
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",https://cdn.akamai.steamstatic.com/steam/apps/1000000/header.jpg?t=1655723048,999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000","List(false, false, true)",27,999,PsychoFlux Entertainment,2021/05/14,0,"ASCENXION is a 2D shoot 'em up game where you explore the field to progress. P

First we will drop the columns that will not help us for the following.

In [0]:
col_to_drop = ['tags', 'header_image', 'platforms', 'short_description', 'type', 'website']

def dropCol(df, col_name):
  return df.drop(*col_name)

df_macro = dropCol(df, col_to_drop)
df_macro.limit(5).toPandas()

,appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age
0,10,"[Multi-player, Valve Anti-Cheat enabled, Onlin...",13990,Valve,0,Action,999,"English, French, German, Italian, Spanish - Sp...",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,999,Valve,2000/11/1,0
1,1000000,"[Single-player, Partial Controller Support, St...",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,999,PsychoFlux Entertainment,2021/05/14,0
2,1000010,"[Single-player, Partial Controller Support, St...",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"Simplified Chinese, English, Japanese, Traditi...",Crown Trick,646,"200,000 .. 500,000",4032,599,"Team17, NEXT Studios",2020/10/16,0
3,1000030,"[Multi-player, Single-player, Co-op, Steam Ach...",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,1999,Vertigo Gaming Inc.,2020/10/14,0
4,1000040,[Single-player],0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,199,DoubleC Games,2019/03/30,0


In [0]:
df_macro.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: string (nullable = true)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)



## 1) The publisher that has released the most games.

In [0]:
game_count = df_macro.groupBy('publisher').agg(F.collect_list('name').alias('games_list'))\
  .withColumn("gameCount", F.size("games_list"))\
  .orderBy(F.desc("gameCount"))

display(game_count.limit(5))

publisher games_list gameCount Big Fish Games List(Fear for Sale: Endless Voyage Collector's Edition, Dark Tales: Edgar Allan Poe's The Premature Burial Collector's Edition, Amaranthine Voyage: The Orb of Purity Collector's Edition, PuppetShow: Porcelain Smile Collector's Edition, Dark Romance: Hunchback of Notre-Dame Collector's Edition, Surface: The Soaring City Collector's Edition, Awakening: The Dreamless Castle, Mystery Trackers: Winterpoint Tragedy Collector's Edition, Mystery of the Ancients: Three Guardians Collector's Edition, Mystery of the Ancients: Deadly Cold Collector's Edition, Grim Facade: Hidden Sins Collector's Edition, Witches' Legacy: The Charleston Curse Collector's Edition, Reflections of Life: Equilibrium Collector's Edition, Mystery Case Files: Key to Ravenhearst Collector's Edition, Grim Tales: The Vengeance Collector's Edition, Fatal Evidence: Cursed Island Collector's Edition, Ms. Holmes: The Monster of the Baskervilles Collector's Edition, Chimeras: Wailing Waters Collector's Edition, The Unseen Fears: Stories Untold Collector's Edition, Redemption Cemetery: Dead Park Collector's Edition, Vermillion Watch: Parisian Pursuit Collector's Edition, Secret City: The Human Threat Collector's Edition, The Man with the Ivory Cane, Dark Tales: Edgar Allan Poe's Ligeia Collector's Edition, Labyrinths of the World: Fool's Gold Collector's Edition, Haunted Legends: The Scars of Lamia Collector's Edition, Mystery Case Files: Moths to a Flame Collector's Edition, Grim Tales: Guest From The Future Collector's Edition, Dark Romance: The Ethereal Gardens Collector's Edition, Dark City: Vienna Collector's Edition, Paranormal Files: Hook Man's Legend Collector's Edition, Myths of the World: The Heart of Desolation Collector's Edition, Halloween Chronicles: Evil Behind a Mask Collector's Edition, Haunted Hotel: Room 18 Collector's Edition, Whispered Secrets: Dreadful Beauty Collector's Edition, Labyrinths of the World: The Wild Side Collector's Edition, Royal Detective: The Last Charm Collector's Edition, Fairy Godmother Stories: Cinderella Collector's Edition, Mystery Case Files: Black Crown Collector's Edition, Christmas Stories: Enchanted Express Collector's Edition, The Christmas Spirit: Grimm Tales Collector's Edition, Secret City: Chalk of Fate Collector's Edition, The Unseen Fears: Ominous Talent Collector's Edition, Living Legends: The Crystal Tear Collector's Edition, Detectives United III: Timeless Voyage Collector's Edition, Immortal Love: Stone Beauty Collector's Edition, Campgrounds IV Collector's Edition, Mindframe: The Secret Design Collector's Edition, Dark City: Dublin Collector's Edition, Fatal Evidence: The Missing Collector's Edition, Dark Romance: Ashville Collector's Edition, Reflections of Life: Utopia Collector's Edition, Nevertales: Hearthbridge Cabinet Collector's Edition, Grim Tales: The Generous Gift Collector's Edition, Mystery Tales: Til Death Collector's Edition, Living Legends Remastered: Ice Rose Collector's Edition, Dark Tales: Edgar Allan Poe's The Bells Collector's Edition, Fright Chasers: Thrills, Chills and Kills Collector's Edition, Ms. Holmes: Five Orange Pips Collector's Edition, Enchanted Kingdom: The Secret of the Golden Lamp Collector's Edition, Chimeras: Cherished Serpent Collector's Edition, PuppetShow: Fatal Mistake Collector's Edition, Haunted Hotel: Lost Time Collector's Edition, Maze: Sinister Play Collector's Edition, Hidden Expedition: The Price of Paradise Collector's Edition, Worlds Align: Deadly Dream Collector's Edition, Fairy Godmother Stories: Dark Deal Collector's Edition, Dark Tales: Edgar Allan Poe's The Devil in the Belfry Collector's Edition, Labyrinths of the World: Hearts of the Planet Collector's Edition, Mystery Case Files: The Harbinger Collector's Edition, Fatal Evidence: Art of Murder Collector's Edition, Dark Romance: Vampire Origins Collector's Edition, Mystery Case Files: Crossfade Collector's Edition, Christmas Stories: The Christmas Tree Forest C

Databricks visualization. Run in Databricks to view.

The publisher that has released the most games is **Big Fish Games** with 422 games.

## 2) What are the best rated game ?

In [0]:
rated_games = df_macro.withColumn('rate_ratio', F.col('positive')/F.col('negative'))
rated_games.select('name', 'rate_ratio')\
    .orderBy(F.desc('rate_ratio'))\
        .limit(20).display()

name,rate_ratio
Flowers -Le volume sur ete-,937.0
Touhou Kishinjou ~ Double Dealing Character.,264.0
Seal World,252.0
Sudocats,247.0
The Upturned,227.0
Don't Escape Trilogy,226.0
From Frontier,219.0
Cleo - a pirate's tale,203.5
Ato,196.0
Zniw Adventure,196.0


Databricks visualization. Run in Databricks to view.

This is the top 20 of the best rated games, based on the ration of positive and negative review. It seems that people love romantical yuri light novel 💓.  
One thing to note : Not over-interpreting that, it is easiest to have a nice ratio with less player than with a lot of player like triple A.

## 3) Years with more released. Impact of covid.

In [0]:
df_macro_date = df_macro.withColumn("release_date_clean", F.to_timestamp(F.col("release_date"), format="y/M/d"))
df_macro_date.drop('release_date')
df_macro_date.limit(5).display()

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age,release_date_clean
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,999,Valve,2000/11/1,0,2000-11-01T00:00:00.000+0000
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,999,PsychoFlux Entertainment,2021/05/14,0,2021-05-14T00:00:00.000+0000
1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil",Crown Trick,646,"200,000 .. 500,000",4032,599,"Team17, NEXT Studios",2020/10/16,0,2020-10-16T00:00:00.000+0000
1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,1999,Vertigo Gaming Inc.,2020/10/14,0,2020-10-14T00:00:00.000+0000
1000040,List(Single-player),0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,199,DoubleC Games,2019/03/30,0,2019-03-30T00:00:00.000+0000


In [0]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, dayofyear, weekofyear

df_macro_date = df_macro_date.withColumn('Years', year(F.col('release_date_clean')))

df_macro_date.limit(5).display()

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age,release_date_clean,Years
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,999,Valve,2000/11/1,0,2000-11-01T00:00:00.000+0000,2000
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,999,PsychoFlux Entertainment,2021/05/14,0,2021-05-14T00:00:00.000+0000,2021
1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil",Crown Trick,646,"200,000 .. 500,000",4032,599,"Team17, NEXT Studios",2020/10/16,0,2020-10-16T00:00:00.000+0000,2020
1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,1999,Vertigo Gaming Inc.,2020/10/14,0,2020-10-14T00:00:00.000+0000,2020
1000040,List(Single-player),0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,199,DoubleC Games,2019/03/30,0,2019-03-30T00:00:00.000+0000,2019


In [0]:
years_releases = df_macro_date.groupBy('Years').agg(F.collect_list('name').alias('games_list'))\
  .withColumn("GameReleasedYear", F.size("games_list"))\
  .orderBy(F.desc("GameReleasedYear"))

display(years_releases.limit(10))

Years games_list GameReleasedYear 2021 List(ASCENXION, Tower of Origin2-Worm's Nest, Hellish Quart, The Undisputables : Online Multiplayer Shooter, Urban Tale, Mars Flight VR, There Was A Dream, Condors Vs Ocelots, Hitchhiker - A Mystery Game, Tetris Effect: Connected, Aether Way, The Truck Game, Pixel Puzzles Illustrations & Anime, Star Drift Evolution, Armored Front: Tiger Command, Here I Come, Totally Reliable Delivery Service, Furs of Fury, Aron's Adventure, Malicious Payload, TASOMACHI: Behind the Twilight, Ynglet, NERVE, Devil Slayer - Raksasi, BOY BEATS WORLD, Chernobylite Enhanced Edition, Miles of Cubes, Gearshifters, DEEEER Simulator: Your Average Everyday Deer Game, Element TD 2 - Tower Defense, Thugs Law, Urizen Frosty Plus The New Warriors, PLAYspace Virtual Music Library, Gravity Heroes, Space Warfare, Rogue Star Rescue, Rick Henderson, Destructive Physics - Destruction Simulator, Flow Weaver, BURIED STARS, Overture Music Visualization, CreatorCrate, Bravery Network Online, DeskScapes, Eruption 爆发, Dis The Game, 30XX, Torque Drift, Going Medieval, Seed of Life, Table of Tales: The Crooked Crown, Farming Life, You Can Kana - Learn Japanese Hiragana & Katakana, Shoot The Zombirds VR, Animal Doctor, Lamentum, The Dawn: Sniper's Way, Hit N' Rush, My Beautiful Paper Smile, METAL DOGS, Zedfest, Dandy Ace, Looking for Heals, Trials of Fire, 拯救大魔王2:逆流 Falsemen2:Upstream, Curious Expedition 2, Dreamscaper, Space Scavenger, World of Aircraft: Glider Simulator, Guts And Goals, Hundred Days - Winemaking Simulator, Desdemona, We Are The Caretakers, Gladiator Guild Manager, Reforged TD - Tower Defense, Alexio, Garden Of Mooj, Mech Mechanic Simulator, Shattered - Tale of the Forgotten King, Action Entities, Helios, Nigate Tale, Rise of Humanity, Knight Squad 2, Run For Cover, Minute of Islands, UFO2: Extraterrestrials, Blocksplode, Skjoldur Story, Toolboy, Soul of Giga, The Red Lantern, Arkio, Logic World, Fling to the Finish, CoronaViruses, Sir Lovelot, Deep Space Battle Simulator, L.S.S II, Home Sweet Home : Survive, Speed Limit, Miner's Mettle, Vain Riser, Hunter's Arena: Legends, Quiet Sleep, Starmancer, Timberborn, Gun Skaters, Garden Story, Suicide For Him, Embr, Dino Lost, New World, Solstice, The Purge Man, Hell Architect, Chess Evolved Online, Crisis Brigade 2 reloaded, Clodhoppers, Dextram, qomp, Partition Sector, Hieroglyph: Prologue, Narita Boy, This Land Is My Land, Ultimate Admiral: Age of Sail, Ultimate Admiral: Dreadnoughts, Rogue Lords, Serin Fate, After I met that catgirl, my questlist got too long!, Tartapolis, Beat Blast, That Flipping Mountain, Curves, Before We Leave, Dust to the End, Cranked Up, Call Of Unity, Weaving Tides, TOHU, Transcender Starship, Banners of Ruin, Viro Move, Alice and You in the planet of numbers, Roguebook, Svoboda 1945: Liberation, Dream Engines: Nomad Cities, Critters for Sale, Motorcycle Mechanic Simulator 2021, Main Assembly, Himno - The Silent Melody, Cris Tales, Where Shadows Slumber, Witchtastic, Gooberries, Side Decide, Fitforce, ASTRO: The Beginning, Before Your Eyes, Viking Vengeance, Sapper - Defuse The Bomb Simulator, SRX, Grow Defense, CountryBalls Heroes, Hindsight 20/20 - Wrath of the Raakshasa, Mister Burnhouse, Crazy World, Yakuza 3 Remastered, Car Mechanic Simulator VR, Rebel Inc: Escalation, Marvel's Guardians of the Galaxy, Aeon Must Die!, JETBOY, Lights Orb, Distant Kingdoms, LithoBreak, Dain Squares, 剑魄 JianPo, Exit From, 月球坠落时 Moon Fall, Rogue Glitch, Inscryption, Piposh, The Wild at Heart, Asterism, Twin Ruin, Sands of Salzaar, Smelter, Royal Crown, Black Legend, Stonedeep, Airtight City 密闭之城1.0, Skellboy Refractured, Fate of WINDSHIFT, Solasta: Crown of the Magister, Mighty Fight Federation, Circuit Superstars, Twelve Minutes, Ezaron Defense, Making it Home, DOOM Eternal: The Ancient Gods - Part Two, The Gardener and the Wild Vines, The Alien Cube, A Total War Saga: TROY, Gamedev Beatdown, Last Contingency, Noara: The Conspiracy, Dragon Call, Deichgraf, ARIS

Databricks visualization. Run in Databricks to view.

We see an incrising of releases between 2013 and 2018. In my opinion it can be explained by the incrising of independent developers, with the incrising of youtube gaming channels that used gaming during these years. Until a developing cap at 2018. It can also be explained by the incrising of technologies development with computers and consoles, that gave more liberty of creation for developers.


After 2018 there are little fluctuations but, in average it seems stabilized. We see no big difference in releasing during the covid (2019-2020).

## 4) How the prizes are distributed ? Many games with discount ?

Firstly we need to convert the price column with actual price. that means pass from 1999 to 19.99 .

In [0]:
df_macro = df_macro.withColumn("price", (F.col("price") / 100).cast("decimal(10,2)"))
df_macro.limit(5).display()

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,999,"English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,9.99,Valve,2000/11/1,0
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"English, Korean, Simplified Chinese",ASCENXION,5,"0 .. 20,000",27,9.99,PsychoFlux Entertainment,2021/05/14,0
1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil",Crown Trick,646,"200,000 .. 500,000",4032,5.99,"Team17, NEXT Studios",2020/10/16,0
1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,English,"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,19.99,Vertigo Gaming Inc.,2020/10/14,0
1000040,List(Single-player),0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,Simplified Chinese,细胞战争,1,"0 .. 20,000",0,1.99,DoubleC Games,2019/03/30,0


In [0]:
df_macro.describe('price').show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|             55691|
|   mean|          7.732850|
| stddev|10.931345827234535|
|    min|              0.00|
|    max|            999.00|
+-------+------------------+



In [0]:
df_macro.groupBy(F.col("price")).count().orderBy(F.col("price")).display()

price,count
0.00,7780
0.28,20
0.29,11
0.30,2
0.31,6
0.37,19
0.38,7
0.39,7
0.41,10
0.44,1


Databricks visualization. Run in Databricks to view.

In [0]:
df_price_dist = df_macro.withColumn(
    "price_category",
    when(F.col("price") < 5, "0-5$")
    .when((F.col("price") < 20) & (F.col("price") > 5), "5-20$")
    .when((F.col("price") < 50) & (F.col("price") > 20), "20-50$")
    .otherwise("50$+")
)

df_price_dist.groupBy("price_category").count().display()

price_category,count
0-5$,31258
20-50$,2628
50$+,353
5-20$,21452


Databricks visualization. Run in Databricks to view.

We clearly see than there are more games under 5 & 20 $ and very few games above 50 $.💲💲💲

Now let's see about the discount.

In [0]:
df_macro.select('discount').count()

Out[23]: 55691

In [0]:
discount_count = df_macro.withColumn('discount_or_not',
    (when(F.col("discount") == "0", "No Discount"))\
    .otherwise("Discount")
)

discount_count.groupBy('discount_or_not').count().display()

discount_or_not,count
No Discount,53173
Discount,2518


Databricks visualization. Run in Databricks to view.

There're much more games without discount than with a discount. 💲

## 5) Most represented language 

In [0]:
df_macro_languages = df_macro.withColumn("languages",F.split(df_macro["languages"], ", "))

df_macro_languages.limit(5).display()

appid,categories,ccu,developer,discount,genre,initialprice,languages,name,negative,owners,positive,price,publisher,release_date,required_age
10,"List(Multi-player, Valve Anti-Cheat enabled, Online PvP, Shared/Split Screen PvP, PvP)",13990,Valve,0,Action,999,"List(English, French, German, Italian, Spanish - Spain, Simplified Chinese, Traditional Chinese, Korean)",Counter-Strike,5199,"10,000,000 .. 20,000,000",201215,9.99,Valve,2000/11/1,0
1000000,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud)",0,IndigoBlue Game Studio,0,"Action, Adventure, Indie",999,"List(English, Korean, Simplified Chinese)",ASCENXION,5,"0 .. 20,000",27,9.99,PsychoFlux Entertainment,2021/05/14,0
1000010,"List(Single-player, Partial Controller Support, Steam Achievements, Steam Cloud, Steam Trading Cards)",99,NEXT Studios,70,"Adventure, Indie, RPG, Strategy",1999,"List(Simplified Chinese, English, Japanese, Traditional Chinese, French, German, Spanish - Spain, Russian, Portuguese - Brazil)",Crown Trick,646,"200,000 .. 500,000",4032,5.99,"Team17, NEXT Studios",2020/10/16,0
1000030,"List(Multi-player, Single-player, Co-op, Steam Achievements, Steam Cloud, Shared/Split Screen, Full controller support, Steam Trading Cards, Shared/Split Screen Co-op, Remote Play on Phone, Remote Play on Tablet, Remote Play on TV, Remote Play Together)",76,Vertigo Gaming Inc.,0,"Action, Indie, Simulation, Strategy",1999,List(English),"Cook, Serve, Delicious! 3?!",115,"100,000 .. 200,000",1575,19.99,Vertigo Gaming Inc.,2020/10/14,0
1000040,List(Single-player),0,DoubleC Games,0,"Action, Casual, Indie, Simulation",199,List(Simplified Chinese),细胞战争,1,"0 .. 20,000",0,1.99,DoubleC Games,2019/03/30,0


In [0]:
df_macro_languages.printSchema()

root
 |-- appid: long (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- ccu: long (nullable = true)
 |-- developer: string (nullable = true)
 |-- discount: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- initialprice: string (nullable = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- name: string (nullable = true)
 |-- negative: long (nullable = true)
 |-- owners: string (nullable = true)
 |-- positive: long (nullable = true)
 |-- price: decimal(10,2) (nullable = true)
 |-- publisher: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- required_age: string (nullable = true)



In [0]:
df_macro_languages = df_macro_languages.select('name', F.explode('languages'))

In [0]:
df_macro_languages = df_macro_languages.withColumnRenamed('col', 'languages')

df_macro_languages.limit(10).display()

name,languages
Counter-Strike,English
Counter-Strike,French
Counter-Strike,German
Counter-Strike,Italian
Counter-Strike,Spanish - Spain
Counter-Strike,Simplified Chinese
Counter-Strike,Traditional Chinese
Counter-Strike,Korean
ASCENXION,English
ASCENXION,Korean


In [0]:
df_macro_languages.columns

Out[29]: ['name', 'languages']

In [0]:
 df_macro_languages\
    .groupBy("languages")\
    .count().withColumnRenamed("count", "game_per_language")\
    .orderBy(F.desc("game_per_language"))\
    .limit(20).display()

languages,game_per_language
English,55116
German,14019
French,13426
Russian,12922
Simplified Chinese,12782
Spanish - Spain,12233
Japanese,10368
Italian,9304
Portuguese - Brazil,6750
Korean,6599


Databricks visualization. Run in Databricks to view.

As we could expected, english is the most represented language in games on the plateform Steam.

## 6) Games prohibited for children (under 16/18)

In [0]:
df_macro.select('required_age').distinct().display()

required_age
7
15
3
16
0
5
18
17
6
180


In [0]:
df_age = df_macro.withColumn(
    "required_age_category",
    when(F.col("required_age") < 16, "<16")
    .when((F.col("required_age") > 16 ), ">16")
    .otherwise("other")
)

df_age.groupBy("required_age_category").count().display()

required_age_category,count
>16,267
other,41
<16,55383


Databricks visualization. Run in Databricks to view.

The majority of games are available for children < 16. Only few games need to have > 16. The other category represents values with a specific character in the value (like a +) contrary at the rest of the values.